In [7]:
import pandas as pd
import numpy as np
from scipy.stats import norm

# Parameters
CL = 0.99
z = 2.3263478740408408  # z-score for 99% confidence
total_value = 1_000_000.0
weights = np.array([0.5, 0.3, 0.2])

# Load returns (daily)
returns = pd.read_csv("sample_data/3assets_returns.csv", index_col=0, parse_dates=True)

# Ensure column order matches weights (and consistent naming)
assets = list(returns.columns)
assert len(assets) == 3, f"Expected 3 assets, found {len(assets)}: {assets}"

# Daily volatilities (sample std)
sigma_assets = returns.std(ddof=1)  # sample std to match covariance.S

# Normal VaR/ES multipliers
k = norm.pdf(z) / (1 - CL)  # ES multiplier

# Asset-level VaR/ES (1-day)
var_1d_assets = z * sigma_assets * total_value
es_1d_assets = k * sigma_assets * total_value

# 10-day scaling (square-root-of-time)
scale_10d = np.sqrt(10)
var_10d_assets = var_1d_assets * scale_10d
es_10d_assets = es_1d_assets * scale_10d

# Portfolio std via covariance matrix
cov_matrix = returns.cov()
sigma_port = np.sqrt(weights @ cov_matrix.values @ weights.T)

# Portfolio VaR/ES
var_1d_port = z * sigma_port * total_value
es_1d_port = k * sigma_port * total_value
var_10d_port = var_1d_port * scale_10d
es_10d_port = es_1d_port * scale_10d

# Assemble results table
results = pd.DataFrame({
    "VaR_1d": pd.concat([var_1d_assets, pd.Series({"Portfolio": var_1d_port})]),
    "ES_1d": pd.concat([es_1d_assets, pd.Series({"Portfolio": es_1d_port})]),
    "VaR_10d": pd.concat([var_10d_assets, pd.Series({"Portfolio": var_10d_port})]),
    "ES_10d": pd.concat([es_10d_assets, pd.Series({"Portfolio": es_10d_port})]),
})

# Reorder rows to Assets then Portfolio
order = assets + ["Portfolio"]
results = results.loc[order]

# Show rounded view for easy reading and also keep full precision in CSV
results_rounded = results.round(2)

print(results_rounded)


# Save to CSV
out_path = "sample_data/var_es_normal_results.csv"
results_rounded.to_csv(out_path)



                  VaR_1d     ES_1d    VaR_10d     ES_10d
Asset_A_Return  27581.71  31599.39   87221.04   99926.05
Asset_B_Return  17208.27  19714.91   54417.33   62344.01
Asset_C_Return  45280.72  51876.52  143190.22  164047.95
Portfolio       20187.01  23127.54   63836.93   73135.71
